In [1]:
import pandas as pd
import numpy as np
import scipy as scp

In [2]:
DATAPATH='data/'

In [4]:
data = pd.read_csv(DATAPATH+'openfoodfacts.csv',delimiter='\t',low_memory=True)

C:\Users\johan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,3,5,24,25,26,28,36,41,42,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(data.shape) # 681'602
print(data["code"].value_counts().count()) # 681533
print(data["manufacturing_places_tags"].value_counts().count()) # 11731
print(data["manufacturing_places"].value_counts().count()) # 12884
print(data["categories"].value_counts().count()) # 62746
print(data["origins_tags"].value_counts().count()) # 6606

data.head()

(665697, 174)
665558
11446
12534
61613
6490


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g,choline_100g,phylloquinone_100g,beta-glucan_100g,inositol_100g,carnitine_100g
0,0000000000017,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1529059080,2018-06-15T10:38:00Z,1529059204,2018-06-15T10:40:04Z,Vitória crackers,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000000031,http://world-en.openfoodfacts.org/product/0000...,isagoofy,1539464774,2018-10-13T21:06:14Z,1539464817,2018-10-13T21:06:57Z,Cacao,NaN,130 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000000000123,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1535737982,2018-08-31T17:53:02Z,1535737986,2018-08-31T17:53:06Z,Sauce Sweety chili 0%,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000291,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1534239669,2018-08-14T09:41:09Z,1534239732,2018-08-14T09:42:12Z,Mendiants,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000949,http://world-en.openfoodfacts.org/product/0000...,kiliweb,1523440813,2018-04-11T10:00:13Z,1523440823,2018-04-11T10:00:23Z,Salade de carottes râpées,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
data['created_t']

0         1529059080
1         1539464774
2         1535737982
3         1534239669
4         1523440813
5         1520506368
6         1537766416
7         1538127563
8         1537883404
9         1517833594
10        1517830801
11        1520419800
12        1532982378
13        1523443867
14        1509478049
15        1511180337
16        1523528250
17        1539629524
18        1522840946
19        1520506122
20        1518439960
21        1539603306
22        1520248076
23        1521725615
24        1538739046
25        1474103866
26        1520597206
27        1528624342
28        1526818055
29        1518691473
             ...    
665667    1422099377
665668    1536926992
665669    1538199223
665670    1536834432
665671    1516747937
665672    1531056615
665673    1534931782
665674    1538245438
665675    1537272417
665676    1530961835
665677    1531473835
665678    1522496106
665679    1527245176
665680    1523963730
665681    1537023318
665682    1538808666
665683    153